Script de desafio de Ciência de Dados

Analisar os dados e desenvolver uma solução
usando Machine Learning visando identificar evasão de alunos.

Necessário instalações:
Jupyter notebook;
Pandas;
Numpy;
Matplotlib;
Seaborn;
Sklearn;

Os modelos utilizados para esse desafio foram "SVM".

obs: Devido a restrições de recursos da máquina, foi necessário utilizar amostragens dos dados durante os processos, visando reduzir a carga no uso do modelo.



In [18]:
#Importações necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report, confusion_matrix,  precision_score, f1_score

from sklearn import svm


In [3]:
#Leitura dos dados 
aluno = pd.read_csv('alunos_teste.csv')
alunoSample = aluno.sample(frac=0.001, random_state=50)


In [4]:
#Leitura dos dados 
provas = pd.read_csv('provas_teste.csv')
provasSample = provas.sample(frac=0.001, random_state=50)


In [5]:
#Leitura dos dados 
acessos = pd.read_csv('acessos_conteudo_teste.csv')
acessosSample = acessos.sample(frac=0.001, random_state=50)
acessos = acessosSample.drop(["DATA_HORA_ACESSO"] ,axis= 1)


In [ ]:
g = sns.PairGrid(acessosSample, diag_sharey=False)
g.map_upper(sns.scatterplot, s=15)
g.map_lower(sns.kdeplot)
g.map_diag(sns.kdeplot, lw=2)

In [6]:
#Agrupando DF de matrículas e disciplinas com o DF de provas;
#Filtrando só a relação entre alunos e o tipo de prova que foi realizado por cada um;

merged_df = aluno.merge( provas, on=['MATRICULA', 'DISCIPLINA'], how='inner')
df =  merged_df.drop(['DATA_HORA_AVALIACAO'], axis =1 )
merged_df = df.drop_duplicates()


In [7]:
#Agrupando dados do resultado do DF acima com o DF de acessos dos conteúdos para relacionar cada acesso com o tipo de prova;

aluno_disciplina = merged_df.merge(acessos,on=['MATRICULA', 'DISCIPLINA'], how='inner')
aluno_disciplina


,MATRICULA,DISCIPLINA,TIPO_AVALIACAO,TEMPO_ACESSO_MINUTOS
0,202009341624,EGT0044,AV,0
1,202009341624,EGT0044,SM,0
2,202007128109,EGT0044,SM,0
3,202007128109,EGT0044,EX,0
4,202007128109,EGT0044,AV,0
...,...,...,...,...
26857,202212133976,ARA1420,SM,9
26858,202212133976,ARA1420,AV,9
26859,202203591452,ARA1420,EX,64
26860,202203591452,ARA1420,SM,64


In [ ]:
sns.jointplot(x ='TEMPO_ACESSO_MINUTOS', y ='TIPO_AVALIACAO', data = aluno_disciplina)

    Exploração dos dados 

In [ ]:
g = sns.PairGrid(aluno_disciplina, diag_sharey=False)
g.map_upper(sns.scatterplot, s=15)
g.map_lower(sns.kdeplot)
g.map_diag(sns.kdeplot, lw=2)

In [ ]:
sns.jointplot(x ='DISCIPLINA', y ='TEMPO_ACESSO_MINUTOS', data = aluno_disciplina)

In [8]:
#Criando coluna de quantidade de disciplinas por aluno e convertendo os valores dos tipos de avaliação em inteiros;

aluno_disciplina['QTT_DISCIPLINAS'] = aluno_disciplina.groupby('MATRICULA')['DISCIPLINA'].transform('nunique')
aluno_disciplina['TIPO_AVALIACAO'], names_tipo = pd.factorize(aluno_disciplina['TIPO_AVALIACAO'])


In [9]:
#Incluindo uma coluna com nova unidade entre a combinação de alunos e disciplinas;

aluno_disciplina['MATRICULA'] = aluno_disciplina['MATRICULA'].astype(str)
aluno_disciplina['ALUNO_DISCIPLINA'] = aluno_disciplina['MATRICULA']+aluno_disciplina['DISCIPLINA']
aluno_disciplina['ALUNO_DISCIPLINA'] , description_names = pd.factorize(aluno_disciplina['ALUNO_DISCIPLINA'])


Com base nos dados, a classificação de uma possível evasão ou não de um aluno se basea em 1 condição;

1- Se o aluno faz a prova "SM" :

obs: testei utilizando outros parametros como "quantidade de disciplinas" e "acessos em minutos" para definir  uma possível evasão do aluno;

In [10]:
def verificar_evasao(row):
    tipo_avaliacao = row['TIPO_AVALIACAO']
       
    if tipo_avaliacao == 1 :
        return 'Não Evadiu'
    else:
        return 'Evadiu'

    
#Analisa uma provável evasão em relação tempo de acesso e quantidade de disciplinas;
def verificar_provavel_evasao(row):
    tipo_avaliacao = row['TIPO_AVALIACAO']
    minutos = row['TEMPO_ACESSO_MINUTOS']
    qtt_disc = row['QTT_DISCIPLINAS']
    
    if tipo_avaliacao == 1 and minutos > 10:
        return 'unlikely'
    if minutos < 20 and qtt_disc < 2:
        return 'verylikely'
    if qtt_disc < 3 and tipo_avaliacao != 1:
        return 'likely'
    else:
        return 'undefined'
    

# Criando a nova coluna "Evasão" com base na coluna "Tipo de Avaliação", indentificando a relação de quem fez a prova SM;
# Podendo testar a outra função "verificar_provavel_evasao";

aluno_disciplina['EVASAO'] = aluno_disciplina.apply(lambda row: verificar_evasao(row), axis=1)
aluno_disciplina

,MATRICULA,DISCIPLINA,TIPO_AVALIACAO,TEMPO_ACESSO_MINUTOS,QTT_DISCIPLINAS,ALUNO_DISCIPLINA,EVASAO
0,202009341624,EGT0044,0,0,1,0,Evadiu
1,202009341624,EGT0044,1,0,1,0,Não Evadiu
2,202007128109,EGT0044,1,0,1,1,Não Evadiu
3,202007128109,EGT0044,2,0,1,1,Evadiu
4,202007128109,EGT0044,0,0,1,1,Evadiu
...,...,...,...,...,...,...,...
26857,202212133976,ARA1420,1,9,1,9539,Não Evadiu
26858,202212133976,ARA1420,0,9,1,9539,Evadiu
26859,202203591452,ARA1420,2,64,4,9540,Evadiu
26860,202203591452,ARA1420,1,64,4,9540,Não Evadiu


      Separação em dados de treinamento e dados de teste

In [11]:
X = aluno_disciplina.drop(["EVASAO","DISCIPLINA","MATRICULA"], axis =1 )
y = aluno_disciplina["EVASAO"]


In [12]:
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(X, y, test_size=0.3, random_state=42)

    Máquinas de Vetores de Suporte (Support Vector Machines - SVM)

In [13]:
# Criar um classificador SVM
clf = svm.SVC(kernel='linear')

In [14]:

# Treinar o modelo com os dados de treinamento
clf.fit(X_treinamento, y_treinamento)



SVC(kernel='linear')

In [15]:
# Fazer previsões com os dados de teste
y_pred = clf.predict(X_teste)

In [17]:
accuracy = accuracy_score(y_teste, y_pred)
print("Precisão: {:.2f}%".format(accuracy * 100))

Precisão: 65.18%
